##### Libraries & Dataset

In [670]:
#IMPORT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('ford.csv')


##### Filtering 

In [671]:
# MODEL COLUMN

df["model"]=df["model"].str.strip()
df["model"].value_counts()
cars=["Escort", "Streetka", "Ranger", "Transit Tourneo", "Fusion"]

df=df[~df["model"].isin(cars)]

df["model"].value_counts()


# YEAR COLUMN

years=[2004, 2006, 2003, 2002, 2004, 1998, 2000, 2060]
df=df[~df["year"].isin(years)]
df["year"].value_counts()

#PRICE

df=df[df["price"]<=42000]

#FUEL TYPE

fuel=["Electric", "Other"]

df=df[~df["fuelType"].isin(fuel)]
df["fuelType"].value_counts()

# TAX

df=df[df["tax"]<=300]

#MPG

df=df[(df["mpg"]<=80) & (df["mpg"]>=28)]

#ENGINE SIZE


df["engineSize"].value_counts()
sizes=[5.0, 1.3, 2.2]
df=df[~df["engineSize"].isin(sizes)]




##### Feature Engineering

In [672]:
# TAX RATE

df["tax_rate"]=(df["tax"]/df["price"])*100
df=df[df["tax_rate"]<=10]

df=df.drop("tax", axis=1)


##### Mapping

In [673]:
#FIX COLUMNS [MODEL]
df["model"]=df["model"].replace(["Fiesta", "KA", "Ka+"], "Hatchback")
df["model"]=df["model"].replace(["Mondeo", "Focus"], "Sedan")
df["model"]=df["model"].replace(["Kuga", "EcoSport", "Puma", "Edge"], "SUV")
df["model"]=df["model"].replace(["C-MAX", "B-MAX", "S-MAX", "Grand C-MAX", "Galaxy"], "MPV")
df["model"]=df["model"].replace(["Tourneo Custom", "Grand Tourneo Connect", "Tourneo Connect"], "Van")
df["model"]=df["model"].replace(["Mustang"], "Coupe")


#### Feature Engineering 2

In [674]:
#PREDICTING CHANCE TO BUY OR NOT
df["model"].value_counts()

def Classify_Cars(row):
    if row["fuelType"]=="Petrol" and row["transmission"]=="Automatic" and row["tax_rate"]<=float(7) and row["mileage"]<=75000:
        if row["model"] in ["Hatchback", "SUV", "Sedan"]:
            if row["price"]<=25000:
                return 0
            else:
                return 1
        else: 
            if row["price"]<=25000:
                return 1
            else: 
                return 2
    
    else:
        return 2

df["car_class"]=df.apply(Classify_Cars, axis=1)



##### Train & Test

In [675]:
#PREDICTING PRICE
from sklearn.model_selection import train_test_split

X=df.drop("car_class", axis=1)
y=df["car_class"]

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

#### Encoding

In [676]:
#ENCODING 2 COLUMNS
from sklearn.preprocessing import OneHotEncoder

cols=["transmission", "fuelType","model"] 
X_train=pd.get_dummies(X_train, columns=cols)
X_test=pd.get_dummies(X_test, columns=cols)



#### Scaling

In [677]:
#SCALING ALL
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

#### KNN

In [678]:
#PREDICTING
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_predict=knn.predict(X_test)

#### Accuracy

In [679]:
#SCORE
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_predict, y_test)
print("The Accuracy Score is:", accuracy)

The Accuracy Score is: 0.9997160704145373
